# Homework 3: Twitter POS tagging

Student Name: Na Chang

Student ID: 858604

Python version used: 2.7

## General info

<b>Due date</b>: 11pm, Sunday April 15

<b>Submission method</b>: see LMS

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day

<b>Marks</b>: 5% of mark for class

<b>Overview</b>: In this homework, you'll be adapting a POS tagger to Twitter data, starting from a tagger trained on Penn Treebank. You will also use prior information on the Twitter tagset to obtain better performance. Finally, you will also analyse your results in a more fine-grained way. For extra credits, you will implement the Expectation-Maximisation algorithm.

<b>Materials</b>: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages; if your iPython notebook doesn't run on the marker's machine, you will lose marks.  

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Extra credit</b>: Each homework has a task which is optional with respect to getting full marks on the assignment, but that can be used to offset any points lost on this or any other homework assignment (but not the final project or the exam). We recommend you skip over this step on your first pass, and come back if you have time: the amount of effort required to receive full marks (1 point) on an extra credit question will be substantially more than earning the same amount of credit on other parts of the homework.

<b>Updates</b>: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

<b>Academic Misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.


### Part 1: Preprocessing (2.0)

<b>Instructions</b>: you first task is to preprocess the data. We will use two datasets for training: 1) the Penn Treebank sample used in the workshops and 2) the Twitter samples data you used in Homework 1. In order to adapt the tagger to the Twitter data we need to built a *joint* vocabulary containing all the word types in PTB and the twitter_samples corpora. So, in addition to preprocessing, your code should also build this vocabulary. Finally, you should also store the tagset, for reasons that will be clearer later.

<b>Important</b>: you are allowed to reuse all the code from the workshop notebooks. In fact you are encouraged to do so as much of this homework will be based on these notebooks.

The vocabulary and the tagset should be stored in Python dictionaries, mapping each word (or tag) to an index (integer). This is similar to what is done in the W6/W7 workshop notebooks. The preprocessed corpora should contain indices only, as in the workshop.

Let's start with the PTB data. You should iterate over all sentences and words, and build the vocabulary and the tagset. Important: make sure you <b>lowercase</b> words before they are added to the dictionary. You should also generate the preprocessed corpus. It should be a list where each element is a tagged sentence, represented as another list of (word, tag) indices (which should correspond to the original words/tags). Print the first preprocessed sentence, the index for the word 'electricity' and the length of the full tagset. (0.5)


In [1]:
import nltk

#vocabulary collection {word:indice}
word_set = {}
#tag collection {tag:indice}
tag_set = {}
#PTB corpus represented by indice pair (word_index, tag_index)
index_tb_corpus = []

nltk.download('treebank')
from nltk.corpus import treebank

tb_corpus = treebank.tagged_sents()
for sent in tb_corpus:
    index_sent = []
    for word, tag in sent:
        windex = word_set.setdefault(word.lower(), len(word_set))
        tindex = tag_set.setdefault(tag, len(tag_set))
        index_sent.append((windex, tindex))
    index_tb_corpus.append(index_sent)

print "Processed TB Sentence 1: \n", index_tb_corpus[0]
print "'electricity' index: \n", word_set['electricity']
print "Tagset length: \n", len(tag_set)

[nltk_data] Downloading package treebank to /Users/Rena/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
Processed TB Sentence 1: 
[(0, 0), (1, 0), (2, 1), (3, 2), (4, 3), (5, 4), (2, 1), (6, 5), (7, 6), (8, 7), (9, 8), (10, 9), (11, 7), (12, 4), (13, 8), (14, 0), (15, 2), (16, 10)]
'electricity' index: 
1095
Tagset length: 
46


<b>Instructions</b>: now you should do the same with the twitter_samples dataset. From now on, we will refer this dataset as the **training** tweets. Since this data is not tagged, the preprocessed corpus should be a list where each element is another list containing indices only (instead of (word, tag) tuples). A tokenised version of twitter_samples is available through the method .tokenized(), use this method to read your corpus. Besides generating the corpus, you should also **update** the vocabulary with the new words from this corpus.

There are two things to keep in mind when doing this process:

1) We will perform a bit more of preprocessing in this dataset, besides lowercasing. Specifically, you should replace special tokens with special symbols, as follows:
- Username mentions are tokens that start with '@': replace these tokens with 'USER_TOKEN'
- Hashtags are tokens that start with '#': replace these with 'HASHTAG_TOKEN'
- Retweets are represented as the token 'RT' (or 'rt' if you lowercase first): replace these with 'RETWEET_TOKEN'
- URLs are tokens that start with 'https://' or 'http://': replace these with 'URL_TOKEN'

2) **Do not create a new vocabulary**. Instead, you should update the vocabulary built from PTB with any new words present in this corpus. These should *include* the special tokens defined above but *not* the original un-preprocessed tokens.

The easiest way to do these steps is by doing 3 passes over the data: preprocess the words first, update the vocabulary and finally convert the corpus into the list format described above. However, it is possible to do all of this in one pass only.

Print the first sentence from your preprocessed corpora, the index for the word 'electricity' and the index for 'HASHTAG_TOKEN'. (0.5)

In [2]:
nltk.download("twitter_samples")
from nltk.corpus import twitter_samples

import re
def process_word(word):
    if re.match(r'@', word):
        return "USER_TOKEN"
    elif re.match(r'#', word):
        return "HASHTAG_TOKEN"
    elif re.match(r'RT', word):
        return "RETWEET_TOKEN"
    elif re.match(r'https?://', word):
        return "URL_TOKEN"
    else:
        return word.lower()

index_tweet_corpus = []
tweet_corpus = twitter_samples.tokenized()
for tweet in tweet_corpus:
    index_sent = []
    for word in tweet:
        pro_word = process_word(word)
        windex = word_set.setdefault(pro_word, len(word_set))
        index_sent.append(windex)
    index_tweet_corpus.append(index_sent)

print "Processed Tweet Training Sentence 1: \n", index_tweet_corpus[0]
print "'electricity' index: \n", word_set['electricity']
print "'HASHTAG_TOKEN' index: \n", word_set['HASHTAG_TOKEN']

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /Users/Rena/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
Processed Tweet Training Sentence 1: 
[11387, 182, 11388, 11389]
'electricity' index: 
1095
'HASHTAG_TOKEN' index: 
11409


<b>Instructions:</b> now we will preprocess the tagged twitter corpus used in W7 (Ritter et al.). This dataset will be referred from now on as **test** tweets. Before you do that though, you should update the tagset.

You might have noticed this in the workshop but this dataset has a few extra tags, besides the PTB ones. These were added to incorporate specific phenomena that happens on Twitter:
- "USR": username mentions
- "HT": hashtags
- "RT": retweets
- "URL": URL addresses

Notice that these special tags correspond to the special tokens we preprocessed before. These steps will be important in Part 3 later.

There a few additional tags which are not specific to Twitter but are not present in the PTB sample:
- "VPP"
- "TD"
- "O"

You should add these new seven tags to the tagset you built when reading the PTB corpus.

Another task is to add an extra type to the vocabulary: `<unk>`. This is in order to account for unknown or out-of-vocabulary words.

Finally, build two "inverted indices" for the vocabulary and the tagset. These should be lists, where the "i"-th element should contain the word (or tag) corresponding to the index "i" in the vocabulary (or tagset).

After doing these tasks, print the index for `<unk>` and the length of your resulting tagset. (0.5)

In [3]:
word_set['<unk>'] = len(word_set)

addi_tags = ['USR', 'HT', 'RT', 'URL', 'VPP', 'TD', 'O']
for tag in addi_tags:
    tag_set[tag] = len(tag_set)

word_inverted = [None] * len(word_set)
counter = 0
for word, index in word_set.items():
    word_inverted[index] = word

tag_inverted = [None] * len(tag_set)
for tag, index in tag_set.items():
    tag_inverted[index] = tag

print "Index for <unk>: \n", word_inverted.index('<unk>')
print "Length of tagset: \n", len(tag_inverted)

Index for <unk>: 
26069
Length of tagset: 
53


<b>Instructions</b>: now we can read the test tweets. Store them in the same format as the PTB corpora (list of lists containing (word, tag) index tuples). Do the same preprocessing steps that you did for the training tweets (lowercasing + replace special tokens). However, **do not** update the vocabulary. Why? Because the test set should simulate a real-world scenario, where out-of-vocabulary words can appear. Instead, after preprocessing each word, you should check if that word is in the vocabulary. If yes, just replace it with its index, otherwise you should replace it with the index for the `<unk>` token. Remember: you can reuse the code from the workshop for this task. Just be mindful that in the workshop we stored words and tags in two separate lists: here you should have a single list, as in the PTB corpus you preprocessed above.

When reading the POS tags for the test tweets you should do some additional preprocessing. There are three tags in this dataset which correspond to PTB tags but are represented with different names:
- "(". In PTB, this is represented as "-LRB-"
- ")". In PTB, this is represented as "-RRB-"
- "NONE". In PTB, this is represented as "-NONE-"

As you build the corpus for the test tweets, you should check if the tag for a word is one of the above. If yes, you should use the PTB equivalent instead. In practice, it is sufficient to ensure you use the correct index for the corresponding tag, using your tagset dictionary. This concept is sometimes referred as *tag harmonisation*, where two different tagsets are mapped to each other.

After this, print the first sentence of your preprocessed corpus. (0.5)

In [4]:
import urllib
try:
    urllib.request.urlretrieve("https://github.com/aritter/twitter_nlp/raw/master/data/annotated/pos.txt","pos.txt")
except: # Python 2
    urllib.urlretrieve("https://github.com/aritter/twitter_nlp/raw/master/data/annotated/pos.txt","pos.txt")

    
def process_tag(tag):
    if tag == '(':
        return '-LRB-'
    elif tag == ')':
        return '-RRB-'
    elif tag == 'NONE':
        return '-NONE-'
    else:
        return tag

index_test_corpus = []

fhdle = open('pos.txt')
index_tweet = []
for line in fhdle:
    if line.strip() == '':
        index_test_corpus.append(index_tweet)
        index_tweet = []
    else:
        word, tag = line.strip().split()
        pro_word = process_word(word)
        if pro_word in word_set.keys():
            windex = word_set[pro_word]
        else:
            windex = word_set['<unk>']
        tindex = tag_set[process_tag(tag)]
        index_tweet.append((windex, tindex))

print "Testing set Sentence 1: \n", index_test_corpus[0]

Testing set Sentence 1: 
[(11392, 46), (61, 19), (114, 11), (8, 7), (3224, 8), (170, 9), (325, 33), (1325, 19), (2375, 22), (3205, 12), (182, 9), (799, 2), (1522, 3), (16, 10), (8490, 0), (1146, 0), (2495, 0), (14038, 43), (26069, 0), (16, 10), (4263, 17), (1760, 4), (9464, 8), (2259, 17), (888, 4), (741, 8), (16, 10)]


<b>Hint</b>: if you did these steps correctly you should have 53 tags in your tagset and around 26000 words in your vocabulary.

### Part 2: Running the PTB tagger on the test tweets (1.5)

<b>Instructions</b>: your next task is to train a POS tagger on the PTB data and try it on the test tweets. This is exactly what we did in W7: feel free to reuse code. However, we are also gonna modify the code a bit.

Your first task is encapsulate the HMM training code into a function. You should name your function `count`. This function should take these input parameters:
- A tagged corpus, in the format described above (list of lists containing (word, tag) index tuples).
- The vocabulary (a dict).
- The tagset (a dict).

Output return values should contain:
- The initial tag probabilities (a vector).
- The transition probabilities (a matrix).
- The emission probabilities (a matrix).

Notice that in the workshop code the vocabulary and tagset were built as part of the training process. Here you should pass them explicitly as parameters instead. This is to ensure our tagger can take into account the words in the training tweets and the extra tags. Important: the workshop code initialise the probabilities with an `eps` value, to ensure you end up with non-zero probabilities for unseen events. You should do the same here.

After writing your function, run it on the PTB corpus to obtain the initial, transition and emission probabilities. (0.5)

In [5]:
import numpy as np

def count(index_corpus, vocabulary, tagset):
    S = len(tagset)
    V = len(vocabulary)

    eps = 0.1

    #pi: initial prob over tags
    #A: transition prob over tags
    #O: ommision prob over word-tag
    pi = eps * np.ones(S)
    A = eps * np.ones((S,S))
    O = eps * np.ones((S, V))

    for sent in index_corpus:
        prev_tag = None
        for word, tag in sent:
            O[tag, word] += 1
            if prev_tag == None:
                pi[tag] += 1
            else:
                A[prev_tag, tag] += 1
            prev_tag = tag
    #normalise prob by dividing with sum
    pi /= np.sum(pi)
    for row in range(S):
        O[row, :] /= np.sum(O[row, :])
        A[row, :] /= np.sum(A[row, :])
    return pi, A, O

hmm_model = count(index_tb_corpus, word_set, tag_set)

<b>Instructions</b>: now you should write a function for Viterbi. The input parameters are the same as in the workshop:
- The parameters (probabilities) of your HMM (a tuple (initial, transition, emission)).
- The input words (a list with numbers).

The output is slightly different though:
- A list of (word, tag) indices, containing the original input word and the predicted tag.

Run Viterbi on the test tweets and store the predictions in a list (might take a few seconds). Remember that in the processing part you stored the test tweets as (word, tag) indices lists: make sure your input to Viterbi are word index lists only. Print the first sentence of your predicted list. (0.5)

In [6]:
def viterbi(params, observations):
    pi, A, O = params
    row = len(observations)
    col = pi.shape[0]

    #table initialise
    #row is observed word
    #col is all tags
    tag_table = np.zeros((row, col))
    tag_table[:,:] = float('-inf')
    backpointers = np.zeros((row, col), 'int')

    #P(tag|<s>)*P(word|tag)
    tag_table[0, :] = pi * O[:, observations[0]]
    #fill in columns
    for r in range(1, row):
        #fill in rows
        for c in range(col):
            #fill in cells
            for csub in range(col):
            #alpha(prev_word, prev_tags)*P(prev_tag, cur_tag)*P(word|cur_tag)
                score = tag_table[r-1, csub] * A[csub, c] * O[c, observations[r]]
                if score > tag_table[r, c]:
                    tag_table[r,c] = score
                    backpointers[r,c] = csub

    #ss: (row, col) from tag_table
    #row: the word index
    #col: the tag index
    ss = []
    #get tag that holds largest score for the last observed word
    ss.append((observations[row-1], np.argmax(tag_table[row-1, :])))
    for i in range(row-1, 0, -1):
        ss.append((observations[i-1], backpointers[i, ss[-1][1]]))

    return list(reversed(ss))

test_tweet = []
for tweet in index_test_corpus:
    tweet_sent = []
    for windex, tindex in tweet:
        tweet_sent.append(windex)
    test_tweet.append(tweet_sent)

#predict tags for index_test_corpus
index_tweet_result = []
for tweet in test_tweet:
    result = viterbi(hmm_model, tweet)
    index_tweet_result.append(result)

print "Predicted Testset Sentence 1: \n", index_tweet_result[0]

Predicted Testset Sentence 1: 
[(11392, 27), (61, 19), (114, 11), (8, 7), (3224, 8), (170, 9), (325, 33), (1325, 19), (2375, 22), (3205, 12), (182, 9), (799, 2), (1522, 3), (16, 10), (8490, 29), (1146, 8), (2495, 8), (14038, 10), (26069, 38), (16, 10), (4263, 29), (1760, 4), (9464, 8), (2259, 17), (888, 4), (741, 8), (16, 10)]


<b>Instructions</b>: you should now evaluate the results. Write a function that takes (word, tag) lists as inputs and outputs the tag sequence using the original tags in the tagset. Your inputs should be a sentence and the tag inverted index you built before.

Run this function on the predictions you obtained above **and** the test tweets, storing them in two separate lists. Finally, flat your predictions into a single list and do the same for the test tweets and report accuracy. (0.5)

In [9]:
def extract_tag(index_sent, tag_inverted):
    tag_sent = []
    for windex, tindex in index_sent:
        tag_sent.append(tag_inverted[tindex])
    return tag_sent

def evaluate(index_tweet_result, index_test_corpus):
    predict_tag = []
    correct_tag = []

    for i in range(len(index_test_corpus)):
        predict_tag.append(extract_tag(index_tweet_result[i], tag_inverted))
        correct_tag.append(extract_tag(index_test_corpus[i], tag_inverted))
    predict_tag = sum(predict_tag, [])
    correct_tag = sum(correct_tag, [])
    return predict_tag, correct_tag


predict_tag, correct_tag = evaluate(index_tweet_result, index_test_corpus)
from sklearn.metrics import accuracy_score
print "Accuracy: ", accuracy_score(correct_tag, predict_tag)

Accuracy:  0.6950938426078367


### Part 3: Adapting the tagger using prior information (1.5)

<b>Instructions</b>: now your task is to adapt the tagger using prior information. What do we mean by that? Remember from part 1 that the twitter tagset has some extra tags, related to special tokens such as mentions and hashtags. In other words, **we know beforehand** that these special tokens **should** have these tags. However, because these tags never appear in the PTB data, the tagger has no such information. We are going to add this in order to improve the tagger.

To recap, we know these things about the twitter data:
- username mentions should have the tag 'USR'
- hashtags should have the tag 'HT'
- retweet tokens should have the tag 'RT'
- URL tokens should have the tag 'URL'

Remember how we replace these tokens with unique special ones (such as 'USER_TOKEN')? Your task is to adapt the emission probabilities for these tokens. Modify the emission matrix: assign 1.0 probability for the emission P('USER_TOKEN'|'USR') and 0.0 for P(word|'USR') for all other words. Do the same for the other three special tags.

In order to do that, you should use the vocabulary and tagset dictionaries in order to obtain the indices for the corresponding words and tags. Then, use the indices to find the values in the emission matrix and modify them. Print your new emission matrix. (0.5)

In [7]:
special_tag = [("USER_TOKEN", "USR"), ("HASHTAG_TOKEN", "HT"), ("RETWEET_TOKEN", "RT"), ("URL_TOKEN", "URL")]
special_tag_index = []
for word, tag in special_tag:
    windex = word_set[word]
    tindex = tag_set[tag]
    special_tag_index.append((tindex, windex))

pi_ptb, A_ptb, O_ptb = hmm_model

for tindex, windex in special_tag_index:
    O_ptb[tindex, :] = 0.0
    O_ptb[tindex, windex] = 1

print "New Emission Matrix: \n", O_ptb

New Emission Matrix: 
[[9.15369893e-05 1.74752434e-04 8.32154448e-06 ... 8.32154448e-06
  8.32154448e-06 8.32154448e-06]
 [1.33457894e-05 1.33457894e-05 6.51955158e-01 ... 1.33457894e-05
  1.33457894e-05 1.33457894e-05]
 [1.62522347e-05 1.62522347e-05 1.62522347e-05 ... 1.62522347e-05
  1.62522347e-05 1.62522347e-05]
 ...
 [3.83582662e-05 3.83582662e-05 3.83582662e-05 ... 3.83582662e-05
  3.83582662e-05 3.83582662e-05]
 [3.83582662e-05 3.83582662e-05 3.83582662e-05 ... 3.83582662e-05
  3.83582662e-05 3.83582662e-05]
 [3.83582662e-05 3.83582662e-05 3.83582662e-05 ... 3.83582662e-05
  3.83582662e-05 3.83582662e-05]]


<b>Instructions</b>: now evaluate your new tagger on the test tweets again. You should report accuracy but also do a fine-grained error analysis. Print the F-scores for **each tag**. <b>Hint:</b> use the "classification_report" function in scikit-learn for that. You should report the tags that performed the best and the worse. (0.5) 

In [10]:
index_tweet_result = []
for tweet in test_tweet:
    result = viterbi(hmm_model, tweet)
    index_tweet_result.append(result)
predict_tag, correct_tag = evaluate(index_tweet_result, index_test_corpus)
from sklearn.metrics import classification_report
print "Accuracy: ", accuracy_score(correct_tag, predict_tag)
print "Classification Report: \n", classification_report(correct_tag, predict_tag)

Accuracy:  0.6950938426078367
Classification Report: 
             precision    recall  f1-score   support

          #       0.00      0.00      0.00         0
          $       0.00      0.00      0.00         0
         ''       0.03      0.20      0.06        91
          ,       0.85      1.00      0.92       303
      -LRB-       0.00      0.00      0.00        32
     -NONE-       0.00      0.00      0.00         2
      -RRB-       0.04      0.15      0.07        34
          .       0.72      0.83      0.77       875
          :       0.97      0.76      0.85       562
         CC       0.96      0.88      0.92       305
         CD       0.59      0.59      0.59       268
         DT       0.74      0.93      0.82       825
         EX       0.38      0.80      0.52        10
         FW       0.00      0.00      0.00         3
         HT       0.98      0.98      0.98       135
         IN       0.81      0.88      0.85      1091
         JJ       0.64      0.59      0.61  

/Library/Python/2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Python/2.7/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


<b>Instructions</b>: finally, based on the information you got above, do some analysis. Why do you think the tagger performed worse on the tags you mentioned above? How would you improve the tagger? Feel free to inspect some instances manually if you want (and show us if you do). Write your analysis in the markdown cell below. Notice that this question is inherently subjective: this is on purpose as you will be evaluated on your analytical abilities. But don't worry about going into depth: 2-4 sentences is enough (but feel free to write more if you need). (0.5)
    

<b>WRITE YOUR ANALYSIS HERE</b>

The tags that performs well are the URL USR HT tags. <br>The ones that are performing badly includes FW, SYM, UH, etc. 

def compare(index_tweet_result, index_test_corpus):<br>
    for i in range(len(index_tweet_result)):<br>
        predict_sent = index_tweet_result[i]
        correct_sent = index_test_corpus[i]

        for j in range(len(predict_sent)):
            word, predict_tag = predict_sent[j]
            word1, correct_tag = correct_sent[j]
            if predict_tag != correct_tag:
                print word_inverted[word], tag_inverted[predict_tag], tag_inverted[correct_tag]

compare(index_tweet_result, index_test_corpus)

Through observations, several reasons causing the low performance:
1. unknow words: For the tags with lowest performance such as UH, SYM etc., many words are marked as unkown in the corpus. 
2. model trained on PTB instead of twitter. The tagger performs poorly on abbreviations, emojis, and other words distinct to tweet.

Improvement
1. Expand the vocabulary to incorporate the different domain words
2. Train the model using corpus from the same domain
3. In addition, can update the emission possiblity to 1 for symbols such as "(" and ")" as in previous section. Implementation as below.

In [12]:
special_tag = [("(", "-LRB-"), (")", "-RRB-"),("'","''"),('"', "''")]
special_tag_index = []
for word, tag in special_tag:
    windex = word_set[word]
    tindex = tag_set[tag]
    special_tag_index.append((tindex, windex))

for tindex, windex in special_tag_index:
    O_ptb[tindex, :] = 0.0
    O_ptb[tindex, windex] = 1

index_tweet_result = []
for tweet in test_tweet:
    result = viterbi(hmm_model, tweet)
    index_tweet_result.append(result)
predict_tag, correct_tag = evaluate(index_tweet_result, index_test_corpus)
from sklearn.metrics import classification_report
print "Accuracy: ", accuracy_score(correct_tag, predict_tag)
print "Classification Report: \n", classification_report(correct_tag, predict_tag)

Accuracy:  0.7087257161672703
Classification Report: 
             precision    recall  f1-score   support

          #       0.00      0.00      0.00         0
          $       0.00      0.00      0.00         0
         ''       1.00      0.73      0.84        91
          ,       0.81      1.00      0.89       303
      -LRB-       0.86      0.97      0.91        32
     -NONE-       0.00      0.00      0.00         2
      -RRB-       0.92      0.97      0.94        34
          .       0.87      0.76      0.81       875
          :       0.94      0.76      0.84       562
         CC       0.96      0.88      0.92       305
         CD       0.53      0.60      0.57       268
         DT       0.74      0.94      0.83       825
         EX       0.35      0.80      0.48        10
         FW       0.00      0.00      0.00         3
         HT       0.98      0.98      0.98       135
         IN       0.81      0.88      0.84      1091
         JJ       0.63      0.61      0.62  

### Extra credits 1: Expectation-Maximisation (1.0)

<b>Instructions</b>: here your goal is to improve the tagger using **hard EM**. This question is divided in two parts. Because EM can take a long time to run we will modify our code above to make it faster and also more robust to underflow by making calculations in the log space.

Your first task is to modify the `count` and `viterbi` functions. For `count`, you should return log probabilities for all matrices. For `viterbi`, you should modify the code in the following way:
- Calculate scores using log probabilities. Remember that in log space, any products become sums. Also remember to make sure you change the base case as well (not only the inner loop).
- You should rewrite the algorithm in vectorised form, in order to make it more efficient. Remember that in Viterbi, the third (inner) `for` loop calculate scores for a single cell, while the second `for` loop calculates scores for a whole column. These operations can be made in parallel, which makes them amenable to vectorisation. Replace the second and third `for` loops in the code with appropriate vector and matrix operations. Remember to do the same to obtain the backpointers. <b>Hint</b>: start by vectorising the inner loop only and check if it is correct.

The second task is to implement EM. This can be done without the first step above but bear in mind that it will take much longer to run. Write a function that:

- 1) Tag a corpus using Viterbi and an initial tagger
- 2) Train a new tagger using the tagged corpora obtained above.
- 3) Repeat both steps above N times.

If you've done the main homework correctly you should be able to reuse the `count` and `viterbi` functions for this and the code should be very straightforward. You should pretrain your tagger using the steps in the main homework, including the tag adaptation in Part 3. Then, in the inner loop, use the training tweets as your unlabelled corpora. Run 5 iterations of EM and report test accuracy **at every iteration**.

EM can take a long time to train, even if you're using the vectorised Viterbi code. If it is too slow in your machine, you're allowed to use a subset of the training tweets for this task.

To get full marks, adapt the algorithm in the following manner:
- At step 2) above, when training the new tagger, combine **both** the PTB gold data and the tagged training tweets, instead of just using the training tweets.

This is an easy trick to obtain better results (and it is essentially **semi-supervised** learning).

In [ ]:
def count(index_corpus, vocabulary, tagset):
    S = len(tagset)
    V = len(vocabulary)

    eps = 0.1

    #pi: initial prob over tags
    #A: transition prob over tags
    #O: ommision prob over word-tag
    pi = eps * np.ones(S)
    A = eps * np.ones((S,S))
    O = eps * np.ones((S, V))

    for sent in index_corpus:
        prev_tag = None
        for word, tag in sent:
            O[tag, word] += 1
            if prev_tag == None:
                pi[tag] += 1
            else:
                A[prev_tag, tag] += 1
            prev_tag = tag
    #normalise prob by dividing with sum
    pi /= np.sum(pi)
    pi = np.log(pi)
    for row in range(S):
        O[row, :] /= np.sum(O[row, :])
        O[row, :] = np.log(O[row, :])
        A[row, :] /= np.sum(A[row, :])
        A[row, :] = np.log(A[row, :])
    
    return pi, A, O


def viterbi(params, observations):
    pi, A, O = params
    #A = np.matrix(A).transpose()
    
    row = len(observations)
    col = pi.shape[0]

    #table initialise
    #row is observed word
    #col is all tags
    tag_table = np.zeros((row, col))
    tag_table[:,:] = float('-inf')
    backpointers = np.zeros((row, col), 'int')

    #P(tag|<s>)*P(word|tag)
    tag_table[0, :] = pi + O[:, observations[0]]
    #fill in columns
    
    for r in range(1, row):
        for c in range(col):
            #fill in cells
            #alpha(prev_word, prev_tags)*P(prev_tag, cur_tag)*P(word|cur_tag)
                arr = tag_table[r-1, :] + A[:, c] + O[c, observations[r]]
                backpointers[r,c] = np.argmax(arr)
                tag_table[r,c] = np.max(arr)
    #ss: (row, col) from tag_table
    #row: the word index
    #col: the tag index
    ss = []
    #get tag that holds largest score for the last observed word
    ss.append((observations[row-1], np.argmax(tag_table[row-1, :])))
    for i in range(row-1, 0, -1):
        ss.append((observations[i-1], backpointers[i, ss[-1][1]]))

    return list(reversed(ss))


def hard_EM_train(hmm_model, training_corpus, ptb_corpus, test_corpus, word_set, tag_set, interation = 5):

	while interation > 1:
		index_tweet_result = []
		for tweet in training_corpus:
		    result = viterbi(hmm_model, tweet)
		    index_tweet_result.append(result)

		for item in ptb_corpus:
			index_tweet_result.append(item)

		new_hmm_model = count(index_tweet_result, word_set, tag_set)

		test_tweet = []
		for tweet in test_corpus:
		    tweet_sent = []
		    for windex, tindex in tweet:
		        tweet_sent.append(windex)
		    test_tweet.append(tweet_sent)
		
		#predict tags for index_test_corpus
		test_index_tweet_result = []
		for tweet in test_tweet:
		    result = viterbi(new_hmm_model, tweet)
		    test_index_tweet_result.append(result)


		predict_tag, correct_tag = evaluate(test_index_tweet_result, index_test_corpus)
		print "Iteration ", iteration, " - Accuracy: ", accuracy_score(correct_tag, predict_tag)

		hard_EM_train(new_hmm_model, training_corpus, ptb_corpus, test_corpus, word_set, tag_set, interation-1)

	return



hard_EM_train(hmm_model, index_tweet_corpus, index_tb_corpus, index_test_corpus, word_set, tag_set)

### Extra credits 2: Soft EM using Forward-Backward (1.0)

<b>Instructions</b>: This is only for the truly intrepid: expect a substantial amount of work to get full marks in this. The goal is to perform **soft EM** using the Forward-Backward algorithm and expected counts. You will need to implement and adapt a set of functions for this task:
- The `count` method, which will still be used for pretraining, needs to also calculate **final** probabilities and return these as part of the output. These are never used in Viterbi but are essential for Forward-Backward.
- Implement the `forward` function. If you want to work in log space you should be careful because it requires summing probabilities. Hint: check the Scipy function `logsumexp` for that. Remember: the algorithm is very similar to Viterbi so feel free to use it as a starting point. The function should return the matrix with the alpha values and the marginal probability for the sentence.
- Implement the `backward` function. This should return a matrix with the beta values and the marginal. Remember the very useful sanity check: for the same sentence and tagger, the marginals returned from `forward` and `backward` should match.
- Implement the `expected_count` function, which is similar to `count` in the sense that it trains a tagger and output new parameters. However, it should have the alpha and beta matrices as inputs, as well as the marginals.

After implementing all of this, rerun EM using the soft approach and evaluate it in a similar way as done in Extra credits 1.
